In [ ]:
!pip install networkx

In [ ]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import ast

### Ler arquivo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
arquivo = '/content/drive/MyDrive/tcc/bases_de_dados/bases_tratadas/base_preprocessada.csv'
df_depressed = pd.read_csv(arquivo, sep = ',', encoding = 'utf-8')

In [ ]:
df_depressed['clean_text'] = df_depressed['clean_text'].apply(ast.literal_eval)
#df_depressed['clean_text'] = df_depressed['clean_text'].apply(lambda x: ' '.join(x))

In [ ]:
df_depressed.head(2)

,coments,created,id,selftext,title,up,full_text,qtd_term,clean_text
0,118.0,1.610661e+09,t3_kxfnzb,"Olá, meu nome é Gabriel, tenho 18 anos e moro ...",Hoje é o dia mais sombrio da minha cidade.,1850.0,"['hoje', 'e', 'o', 'dia', 'mais', 'sombrio', '...",6,"[dia, sombrio, cidade, nome, ano, morar, manau..."
1,808.0,1.658598e+09,t3_w69jxb,"Em 2019 reencontrei minha primeira namorada, v...",Meu casamento me destruiu financeiramente.,1558.0,"['meu', 'casamento', 'me', 'destruiu', 'financ...",6,"[casamento, primeiro, ano, relacionamento, pat..."


### text graph rank para encontrar as palavras mais significativas por texto

In [ ]:
def get_vocab(topics_text):
  """Get all tokens"""
  vocab_list = []
  for text in topics_text:
    vocab = OrderedDict()
    i = 0
    for word in text:
        if word not in vocab:
            vocab[word] = i
            i += 1
    vocab_list.append(vocab)
  return vocab_list

In [ ]:
def get_token_pairs(window_size, texts):
  """Build token_pairs from windows in texts"""
  tokens_list = []
  for text in texts:
    token_pairs = []
    for i, word in enumerate(text):
        for j in range(i+1, i+window_size):
            if j >= len(text):
                break
            pair = (word, text[j])
            if pair not in token_pairs:
                token_pairs.append(pair)
    tokens_list.append(token_pairs)
  return tokens_list

In [ ]:
def get_matrix(vocab_list, tokens_list):
  """Get normalized matrix"""
  # Build matrix
  matrix_list = []
  for index, vocab in enumerate(vocab_list):
    vocab_size = len(vocab)
    matrix = np.zeros((vocab_size, vocab_size), dtype='float')
    for word1, word2 in tokens_list[index]:
        i, j = vocab[word1], vocab[word2]
        matrix[i][j] = 1

    # Get Symmeric matrix
    matrix = matrix + matrix.T - np.diag(matrix.diagonal())

    # Normalize matrix by column
    norm = np.sum(matrix, axis=0)
    matrix_norm = np.divide(matrix, norm, where=norm!=0) # this is ignore the 0 element in norm
    matrix_list.append(matrix_norm)
  return matrix_list

In [ ]:
# Iteration
def epoch_interation(vocab_list, matrix_list, steps, damping, min_diff):
  score_matrix_list = []
  for index, vocab in enumerate(vocab_list):
    score_matrix = np.array([1] * len(vocab))
    previous_score = 0
    for epoch in range(steps):
        score_matrix = (1-damping) + damping * np.dot(matrix_list[index], score_matrix)
        if abs(previous_score - sum(score_matrix))  < min_diff:
            break
        else:
            previous_score = sum(score_matrix)
    score_matrix_list.append(score_matrix)
  return score_matrix_list

In [ ]:
# Get weight for each node
def get_weight(score_matrix_list, vocab_list):
  node_weight_list = []
  for general_index, vocab in enumerate(vocab_list):
    node_weight = dict()
    for word, word_index in vocab.items():
        node_weight[word] = score_matrix_list[general_index][word_index]
    node_weight_list.append(node_weight)
  return node_weight_list

In [ ]:
def get_keywords(node_weight_list):
  keywords_dict = {}
  for node_weight in node_weight_list:
    for key, value in node_weight.items():
      if key in keywords_dict.keys():
        keywords_dict[key] += value
      else:
        keywords_dict[key] = value
    # Ordena os itens do dicionário por valor (matriz de adjacência) em ordem decrescente
    keywords_dict = dict(sorted(keywords_dict.items(), key=lambda x: x[1], reverse=True))
  return keywords_dict

# Utilizando as funções

In [ ]:
#Criar vocabulário
vocab_list = get_vocab(df_depressed['clean_text'])

In [ ]:
#Criar pares de coocorrencia
tokens_list = get_token_pairs(5, df_depressed['clean_text'])

In [ ]:
#Criar matriz de similaridade
matrix_list = get_matrix(vocab_list, tokens_list)

In [ ]:
#Iteração de pesos dos vértices
steps = 10
damping = 0.85 # damping coefficient, usually is .85
min_diff = 1e-5 # convergence threshold
score_matrix_list = epoch_interation(vocab_list, matrix_list, steps, damping, min_diff)

In [ ]:
#Coleta dos pesos dos vértices anexados a palavra por texto
node_weight_list = get_weight(score_matrix_list, vocab_list)

In [ ]:
#Soma da probabilidade de relevancia de todas as palavras iguais de todos os textos
keywords_dict = get_keywords(node_weight_list)

In [ ]:
len(keywords_dict)

4283

# Grounded theory

In [ ]:
#Retirar palavras do dicionario
def remove_words(dictionary, not_words):
    return {key: value for key, value in dictionary.items() if key not in not_words}

In [ ]:
#Pegar as palavras com os 100 maiores valores
def top_values(keywords_dict, number):
    # Use a função sorted para classificar o dicionário com base nos valores em ordem decrescente
    sorted_itens = sorted(keywords_dict.items(), key=lambda x: x[1], reverse=True)

    # Pegue os cem primeiros pares chave-valor
    top_words = sorted_itens[:number]

    # Converta a lista de pares em um novo dicionário
    top_dict = dict(top_words)

    return top_dict

In [ ]:
not_words = ['primeiro', 'segundo', 'mesmo', 'situação', 'coisa', 'fato', 'vez', 'grande', 'pequeno', 'ponto', 'sao', 'voce',
             'carar', 'causa', 'motivo', 'cabecar']
keywords_dict = remove_words(keywords_dict, not_words)

In [ ]:
top_keywords = top_values(keywords_dict, 100)

In [ ]:
'''#Alternativa: pegar só palavras com frequência acima de x desvios padrão

mean = np.mean(list(freq_dict.values()))
std = np.std(list(freq_dict.values()))

print(mean,std)

threshold = mean+4*std

freq_words = []
for word, freq in freq_dict.items():
  if freq > threshold:
    freq_words.append(word)

freq_keywords = list(set(freq_words) & set(keywords_list))'''

'#Alternativa: pegar só palavras com frequência acima de x desvios padrão\n\nmean = np.mean(list(freq_dict.values()))\nstd = np.std(list(freq_dict.values()))\n\nprint(mean,std)\n\nthreshold = mean+4*std\n\nfreq_words = []\nfor word, freq in freq_dict.items():\n  if freq > threshold:\n    freq_words.append(word)\n\nfreq_keywords = list(set(freq_words) & set(keywords_list))'

In [ ]:
top_keywords.keys()

dict_keys(['dia', 'pessoa', 'vida', 'ano', 'tempo', 'pai', 'bom', 'casa', 'amigo', 'gente', 'problema', 'trabalho', 'Mundo', 'momento', 'novo', 'hora', 'mulher', 'semana', 'medo', 'forma', 'relacionamento', 'consigo', 'mau', 'mês', 'feliz', 'dinheiro', 'Escola', 'vontade', 'lugar', 'homem', 'dor', 'atra', 'mensagem', 'noite', 'situacao', 'Faculdade', 'corpo', 'dificil', 'ruim', 'verdade', 'sentimento', 'final', 'irmao', 'desabafo', 'voltar', 'filho', 'foto', 'historio', 'depressao', 'lar', 'conversa', 'ansiedade', 'amig', 'passado', 'sozinho', 'emprego', 'unico', 'jeito', 'social', 'garota', 'diferente', 'feio', 'pessoal', 'desculpa', 'fim', 'poder', 'normal', 'crianca', 'amor', 'quarto', 'falta', 'terapia', 'amizade', 'possivel', 'contato', 'varia', 'amo', 'pau', 'ideiar', 'sozinha', 'odeio', 'junto', 'vergonha', 'cheio', 'lembro', 'raiva', 'idade', 'experiencia', 'olho', 'legal', 'serio', 'velho', 'preciso', 'fisico', 'medico', 'ultimo', 'assunto', 'jogo', 'estranho', 'longo'])

In [ ]:
#'vida', 'bom',  'coisa',  'vez', 'carar', 'problema',  'Mundo',  'novo', 'grande', 'forma',  'consigo', 'mau', 'mesmo',  'lugar',   'primeiro', 'pequeno', 'causa', 'atra',  'voce', 'fato', 'situacao', 'ruim', 'motivo', 'verdade', 'desabafo', 'voltar',  'ponto', 'foto', 'historio', 'depressao', 'ansiedade', 'unico', 'jeito',  'diferente',  'pessoal',  'poder', 'normal', 'quarto', , 'terapia',  'possivel',  'varia',  'ideia', 'junto' , 'lembro',

In [ ]:
tempo = ['dia', 'ano', 'tempo', 'momento', 'hora', 'semana', 'mês', 'passado', 'segundo', 'noite', 'fim', 'final', 'idade', 'ultimo']
familia = ['pai', 'casa', 'relacionamento', 'irmao', 'filho', 'lar', 'crianca']
labor_sustento = ['trabalho', 'Escola', 'Faculdade', 'emprego', 'dinheiro']
corpo_fisico = ['cabecar', 'dor', 'corpo', 'feio', 'pau', 'fisico', 'medico', 'velho']
relacionamentos = ['pessoa', 'amigo', 'amig', 'gente','relacionamento', 'conversa', 'social', 'garota','amor', 'amizade', 'contato', 'homem', 'mulher', 'mensagem']
emocoes = ['raiva', 'vergonha', 'sozinha', 'sozinho', 'desculpa', 'odeio', 'amo', 'sentimento', 'feliz', 'medo', 'vontade','falta', 'dificil', 'cheio', 'preciso']

In [ ]:
categorys_dict = {'tempo': tempo, 'familia': familia, 'labor_sustento' : labor_sustento, 'corpo_fisico' : corpo_fisico, 'relacionamentos':relacionamentos}

In [ ]:
# Selecionar textos com as palavras da categoria escolhida
def category_split(df, column, categorys_dict):
  category_dfs = {}
  for category, words_list in categorys_dict.items():
    print(category)
    df_category = pd.DataFrame()
    for index, clean_text in enumerate(df[column]):
      for word in words_list:
        if word in clean_text and index not in df_category.index:
         # print(word)
          df_category = pd.concat([df_category, df.iloc[index:]])
          break

    category_dfs[category] = df_category
  return category_dfs

In [ ]:
categorys_dfs = category_split(df_depressed, 'full_text', categorys_dict)

tempo
familia
labor_sustento
corpo_fisico
relacionamentos


In [ ]:
list(categorys_dfs.values())[0]['selftext']

0      Olá, meu nome é Gabriel, tenho 18 anos e moro ...
1      Em 2019 reencontrei minha primeira namorada, v...
2      Sou médico anestesista, trabalho na linha de f...
3      Sexo é vida, né? Acho que todos que leram algu...
4      Eu já vi alguns desabafos aqui sobre a clássic...
                             ...                        
262    sempre fui magro e bonitinho, nunca o mais lin...
263     as pessoas se aproximam, curiosas, atentas, e...
264    ontem em uma festa tive uma recaída muito gran...
265     Velho, estou com umas tretas psicológicas, es...
266    Boa noite pessoal, me chamo Rafael, tenho 18 a...
Name: selftext, Length: 267, dtype: object

In [ ]:
len(df_depressed)

267

In [ ]:
categorys_dfs['labor_sustento'].head(2)

,coments,created,id,selftext,title,up,full_text,qtd_term,clean_text
1,808.0,1.658598e+09,t3_w69jxb,"Em 2019 reencontrei minha primeira namorada, v...",Meu casamento me destruiu financeiramente.,1558.0,"['meu', 'casamento', 'me', 'destruiu', 'financ...",6,"[casamento, primeiro, ano, relacionamento, pat..."
2,148.0,1.616290e+09,t3_m9l8o0,"Sou médico anestesista, trabalho na linha de f...",Sabe o real motivo pelo qual você deveria fica...,1517.0,"['sabe', 'o', 'real', 'motivo', 'pelo', 'qual'...",7,"[Real, motivo, casa, medico, anestesisto, trab..."


In [ ]:
print(f"tempo: {len(categorys_dfs['tempo'])} \n familia: {len(categorys_dfs['familia'])} \n labor_sustento: {len(categorys_dfs['labor_sustento'])} \n corpo_fisico: {len(categorys_dfs['corpo_fisico'])} \n relacionamentos: {len(categorys_dfs['relacionamentos'])}")

tempo: 267 
 familia: 267 
 labor_sustento: 266 
 corpo_fisico: 266 
 relacionamentos: 267


In [ ]:
node_weight_list = {key:value for key, value in node_weight_list.items() if key in labor_sustento}
vocab_list =
top_keywords =

In [ ]:
top_keywords

{'dia': 269.4153725467066,
 'pessoa': 253.42214268320498,
 'vida': 248.26766402087813,
 'ano': 239.10590178135686,
 'tempo': 183.42903263377303,
 'pai': 181.51430241149916,
 'bom': 159.00524916399974,
 'casa': 157.9766026794519,
 'amigo': 128.98729265140355,
 'gente': 123.41614541409359,
 'problema': 98.07886790196807,
 'trabalho': 89.71480822303022,
 'Mundo': 85.87177471761144,
 'momento': 85.83615398780232,
 'novo': 85.77237930200576,
 'hora': 80.05660096797115,
 'mulher': 75.18147132987283,
 'semana': 70.84455041468738,
 'medo': 70.83284026467796,
 'forma': 69.11422714155903,
 'relacionamento': 68.8522271560703,
 'consigo': 68.04821851846313,
 'mau': 65.54546612630368,
 'mês': 61.09796679930442,
 'feliz': 60.181916029915435,
 'dinheiro': 59.62542185259543,
 'Escola': 59.07342861198516,
 'vontade': 55.54395213996569,
 'lugar': 52.71565171966456,
 'homem': 52.565754359769166,
 'dor': 51.12894043443313,
 'atra': 47.73104608888582,
 'mensagem': 47.4739600639507,
 'noite': 46.75366480203

In [ ]:
df_category

,coments,created,id,selftext,title,up,full_text,qtd_term,clean_text
0,118.0,1.610661e+09,t3_kxfnzb,"Olá, meu nome é Gabriel, tenho 18 anos e moro ...",Hoje é o dia mais sombrio da minha cidade.,1850.0,"['hoje', 'e', 'o', 'dia', 'mais', 'sombrio', '...",6,"[dia, sombrio, cidade, nome, ano, morar, manau..."
0,118.0,1.610661e+09,t3_kxfnzb,"Olá, meu nome é Gabriel, tenho 18 anos e moro ...",Hoje é o dia mais sombrio da minha cidade.,1850.0,"['hoje', 'e', 'o', 'dia', 'mais', 'sombrio', '...",6,"[dia, sombrio, cidade, nome, ano, morar, manau..."
0,118.0,1.610661e+09,t3_kxfnzb,"Olá, meu nome é Gabriel, tenho 18 anos e moro ...",Hoje é o dia mais sombrio da minha cidade.,1850.0,"['hoje', 'e', 'o', 'dia', 'mais', 'sombrio', '...",6,"[dia, sombrio, cidade, nome, ano, morar, manau..."
0,118.0,1.610661e+09,t3_kxfnzb,"Olá, meu nome é Gabriel, tenho 18 anos e moro ...",Hoje é o dia mais sombrio da minha cidade.,1850.0,"['hoje', 'e', 'o', 'dia', 'mais', 'sombrio', '...",6,"[dia, sombrio, cidade, nome, ano, morar, manau..."
1,808.0,1.658598e+09,t3_w69jxb,"Em 2019 reencontrei minha primeira namorada, v...",Meu casamento me destruiu financeiramente.,1558.0,"['meu', 'casamento', 'me', 'destruiu', 'financ...",6,"[casamento, primeiro, ano, relacionamento, pat..."
...,...,...,...,...,...,...,...,...,...
257,5.0,1.680473e+09,t3_129zrrk,"\n\nOi gente, tudo bem? Bom...Sou apenas uma ...",Histórias Tristes De Aniversário,3.0,"['historias', 'tristes', 'de', 'aniversario', ...",7,"[historia, triste, aniversario, gente, bom, am..."
258,1.0,1.680490e+09,t3_12a7749,Eu desde criança sempre fui muito responsável ...,Estou cansado de me esforçar por quem não me d...,1.0,"['estou', 'cansado', 'de', 'me', 'esforcar', '...",18,"[valor, humilha, crianca, responsavel, ano, id..."
259,2.0,1.680489e+09,t3_12a6mqy,"Eu não aguento mais, não aguento viver nesse c...",Desabafo:,1.0,"['desabafo', '', 'eu', 'nao', 'aguento', 'mais...",7,"[desabafo, aguentar, aguentar, corpo, poder, p..."
260,1.0,1.680488e+09,t3_12a6gb2,"\nEntão pessoal, 4 meses atrás conheci essa me...",Término Traumático,1.0,"['termino', 'traumatico', '', 'entao', 'pessoa...",11,"[traumatico, pessoal, mês, atra, Menina, bumbl..."


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from collections import OrderedDict

def plot_graph(node_weight_list, vocab_list, keywords_list):
    G = nx.Graph()

    # Cria um dicionário para associar cada palavra ao seu peso médio
    # (Dani): Se colocar o if word in keywords list antes das coisas, você só vai prpecisar calcular pesos em quem de fato vai ser plotado!
    word_weight_dict = {}
    for node_weight in node_weight_list:
        for word, weight_array in node_weight.items():
            # Calcula a média dos pesos para atribuir ao nó
            weight = sum(weight_array) / len(weight_array)
            word_weight_dict[word] = weight
            if word in keywords_list:
                G.add_node(word, weight=weight)

    # Adiciona as arestas com base no vocab_list
    for vocab_dict in vocab_list:
        for word, _ in vocab_dict.items():
            if word in G.nodes():
                G.add_edge(list(vocab_dict.keys())[0], word)

    # Obtém as posições dos nós para o layout do grafo
    pos = nx.spring_layout(G)

    # Obtém os pesos dos nós para atribuir ao tamanho dos nós no gráfico
    node_weights = nx.get_node_attributes(G, 'weight')
    node_sizes = [10000 * word_weight_dict[word] for word in G.nodes()]

    # Desenha os nós e arestas do grafo
    plt.figure(figsize=(20, 20))
    nx.draw(G, pos, with_labels=True, node_size=node_sizes, font_size=8)
    plt.show()


In [ ]:
plot_graph(node_weight_list, vocab_list, freq_keywords)

NameError: ignored